In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\mlOps\\End-to-End-Machine-Learning-Pipeline'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from my_project.constants import *
from my_project.utils.common import read_yaml, create_directories

In [8]:
%pwd

'd:\\mlOps\\End-to-End-Machine-Learning-Pipeline'

In [9]:
print(os.getcwd())
CONFIG_FILE_PATH

d:\mlOps\End-to-End-Machine-Learning-Pipeline


WindowsPath('config/config.yaml')

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from my_project import logger
from my_project.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-20 18:57:09,561]: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-20 18:57:09,566]: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 18:57:09,568]: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-20 18:57:09,569]: INFO: common: created directory at: artifacts]
[2025-09-20 18:57:09,570]: INFO: common: created directory at: artifacts/data_ingestion]
[2025-09-20 18:57:10,525]: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21034
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "444bee1bcf0e666f94f4a9f1e4b3635e03b8b273ac48845f382273d098afef12"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 179E:3CA177:A83FBA:C68FD4:68CEDCE5
Accept-Ranges: bytes
Date: S